In [2]:
'''
@Author: Viney Khaneja
@Date: 2022-05-21 09:13
@Last Modified by: Viney Khaneja
@Last Modified time: None
@Title : CPU Logs PFiles Loading from HDFS using PySpark DataFrame
'''

from pyspark.sql import *
# spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import *
spark = SparkSession.builder.config("spark.jars", "/usr/share/java/mysql-connector-java-8.0.29.jar").master("local").appName("PySpark_MySQL_test").getOrCreate()

22/05/24 13:05:26 WARN Utils: Your hostname, viney-Lenovo-G50-80 resolves to a loopback address: 127.0.1.1; using 192.168.0.196 instead (on interface wlp3s0)
22/05/24 13:05:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/24 13:05:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Loading CPU Log Files from Hadoop in Pyspark DataFrame

In [3]:
rdd_cpu_logs_data_hdfs = spark.read.csv("hdfs://localhost:9000/cpulogs/*.csv",header=True)

In [4]:
df1 = rdd_cpu_logs_data_hdfs.select(["user_name","DateTime","keyboard","mouse",'Cpu Count'])
df2 = df1.groupBy("user_name").count()
rdd_cpu_logs_data_hdfs.createOrReplaceTempView("tbl_working_counts")

22/05/24 13:07:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
hdfs_pyspark_df = rdd_cpu_logs_data_hdfs.select(["user_name","DateTime","keyboard","mouse",'Cpu Count'])
hdfs_pyspark_df.show()
hdfs_pyspark_df.createOrReplaceTempView("tbl_sql_try")

+--------------------+-------------------+--------+------+---------+
|           user_name|           DateTime|keyboard| mouse|Cpu Count|
+--------------------+-------------------+--------+------+---------+
|  iamnzm@outlook.com|2019-09-19 08:40:02|     1.0|  32.0|        2|
|  iamnzm@outlook.com|2019-09-19 08:45:02|     0.0|   0.0|        2|
|  iamnzm@outlook.com|2019-09-19 08:50:01|     0.0|   0.0|        2|
|  iamnzm@outlook.com|2019-09-19 08:55:01|    11.0| 900.0|        2|
|  iamnzm@outlook.com|2019-09-19 09:00:01|     2.0|  25.0|        2|
|  iamnzm@outlook.com|2019-09-19 09:05:01|    37.0| 336.0|        2|
|deepshukla292@gma...|2019-09-19 09:05:01|     0.0|  55.0|        2|
|  iamnzm@outlook.com|2019-09-19 09:10:01|     0.0| 136.0|        2|
|deepshukla292@gma...|2019-09-19 09:10:01|     6.0|1112.0|        2|
|  iamnzm@outlook.com|2019-09-19 09:15:02|     0.0|  84.0|        2|
|deepshukla292@gma...|2019-09-19 09:15:02|     0.0| 357.0|        2|
|markfernandes66@g...|2019-09-19 0

In [6]:
import os
database = os.getenv('MYSQL_DB_NAME')
user= os.getenv('MYSQL_DB_USER')
password = os.getenv('MYSQL_PASSWORD')
host= os.getenv('MYSQL_HOST')
print(database)


cpulogdata


In [7]:
try:
    jdb_curl = f"jdbc:mysql://localhost/"+database
    hdfs_pyspark_df.write.format("jdbc").option("url",jdb_curl) \
        .mode("overwrite")\
        .option("dbtable","tbl_sql_try") \
        .option("driver","com.mysql.jdbc.Driver")\
        .option("user",user).option("password",password).save()
    print("Data uploaded to MySql")
except Exception as ex:
    print(ex)

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


Data uploaded to MySql
